In [52]:
!python -V

Python 3.9.19


In [53]:
import pandas as pd

In [54]:
import pickle

In [55]:
import seaborn as sns
import matplotlib.pyplot as plt

In [56]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [62]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment-test")

2024/05/27 14:01:50 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment-test' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/amohan/personal/anj/personal-projects/mlopszoomcamp/mlopszoomcamp2024/github-codebase/github/mlopszoomcamp-2024/week2-experiment-tracking/mlruns/6', creation_time=1716832910708, experiment_id='6', last_update_time=1716832910708, lifecycle_stage='active', name='nyc-taxi-experiment-test', tags={}>

In [35]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    #df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    #df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [36]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [37]:
len(df_train), len(df_val)

(73908, 61921)

In [38]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [39]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [40]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [41]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715200888857

In [42]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [43]:
with mlflow.start_run():

    mlflow.set_tag("developer", "amohan")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [44]:
import xgboost as xgb

In [45]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [46]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [47]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [48]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:08:46] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.43408                          
[1]	validation-rmse:9.18924                           
[2]	validation-rmse:8.33796                           
[3]	validation-rmse:7.76897                           
[4]	validation-rmse:7.39218                           
[5]	validation-rmse:7.14349                           
[6]	validation-rmse:6.97752                           
[7]	validation-rmse:6.86548                           
[8]	validation-rmse:6.78901                           
[9]	validation-rmse:6.73676                           
[10]	validation-rmse:6.69927                          
[11]	validation-rmse:6.67234                          
[12]	validation-rmse:6.65347                          
[13]	validation-rmse:6.63849                          
[14]	validation-rmse:6.62794                          
[15]	validation-rmse:6.61858                          
[16]	validation-rmse:6.61119                          
[17]	validation-rmse:6.60364                          
[18]	valid

/opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:10:01] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.69076                                                      
[1]	validation-rmse:6.85728                                                      
[2]	validation-rmse:6.68502                                                      
[3]	validation-rmse:6.63247                                                      
[4]	validation-rmse:6.60529                                                      
[5]	validation-rmse:6.58479                                                      
[6]	validation-rmse:6.57554                                                      
[7]	validation-rmse:6.57072                                                      
[8]	validation-rmse:6.56311                                                      
[9]	validation-rmse:6.56146                                                      
[10]	validation-rmse:6.55872                                                     
[11]	validation-rmse:6.55765                                                     
[12]	validation-

...

/opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:02:40] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.27537                                                    
[1]	validation-rmse:10.47747                                                    
[2]	validation-rmse:9.80375                                                     
[3]	validation-rmse:9.23783                                                     
[4]	validation-rmse:8.76248                                                     
[5]	validation-rmse:8.36850                                                     
[6]	validation-rmse:8.04343                                                     
[7]	validation-rmse:7.77218                                                     
[8]	validation-rmse:7.55115                                                     
[9]	validation-rmse:7.36771                                                     
[10]	validation-rmse:7.21989                                                    
[11]	validation-rmse:7.09524                                                    
[12]	validation-rmse:6.99480

/opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:04:02] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.62059                                                    
[1]	validation-rmse:11.08384                                                    
[2]	validation-rmse:10.59853                                                    
[3]	validation-rmse:10.16068                                                    
[4]	validation-rmse:9.76628                                                     
[5]	validation-rmse:9.41222                                                     
[6]	validation-rmse:9.09472                                                     
[7]	validation-rmse:8.81052                                                     
[8]	validation-rmse:8.55658                                                     
[9]	validation-rmse:8.33036                                                     
[10]	validation-rmse:8.12806                                                    
[11]	validation-rmse:7.94831                                                    
[12]	validation-rmse:7.78840

/opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [49]:
mlflow.xgboost.autolog(disable=True)

In [50]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:49:01] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

/opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:50:11] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [51]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2024/05/27 12:50:40 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/27 12:50:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."
2024/05/27 12:50:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlflow-experimenttracking/li

AssertionError: /opt/anaconda3/envs/mlflow-experimenttracking/lib/python3.9/distutils/core.py